# Chart Events for last 48 period

TODO:

1. [ ] Create or find list of `ITEMID`s that correlate to the 17 dimmensions from paper.
2. [ ] Extract the 17 dims from `CHARTEVENTS.csv` for each sample

|	Chart Event	|	Dim	|	Normal	|
| --- | :--- | :--- |
|	1. Glasgow coma scale eye opening	|	4	|	4 Spontaneously	|
|	2. Glasgow coma scale verbal response	|	5	|	5 Oriented	|
|	3. Glasgow coma scale motor response	|	6	|	6 Obeys Commands	|
|	4. Glasgow coma scale total	|	13	|	15	|
|	5. Capillary refill rate	|	2	|	Normal < 3 secs	|
|	6. Diastolic blood pressure	|	1	|	70	|
|	7. Systolic blood pressure	|	1	|	105	|
|	8. Mean blood pressure	|	1	|	87.5	|
|	9. Heart Rate	|	1	|	80	|
|	10. Glucose	|	1	|	85	|
|	11. Fraction inspired oxygen	|	1	|	0.21	|
|	12. Oxygen saturation	|	1	|	97.5	|
|	13. Respiratory rate	|	1	|	15	|
|	14. Body Temperature	|	1	|	37	|
|	15. pH	|	1	|	7.4	|
|	16. Weight	|	1	|	80.7	|
|	17. Height	|	1	|	168.8	|

![Glascow](assets/images/GCS.jpg)

https://www.firstaidforfree.com/glasgow-coma-scale-gcs-first-aiders/

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
import traceback
import os
import dask.dataframe as dd
from dask.distributed import Client

In [3]:
CHARTEVENTS_FILENAME = 'mimic-iii/CHARTEVENTS.csv'
READMISSION_FILENAME = 'data/readmission.csv'
SAMPLES_DIR = 'data/parquet/'
DATASTORE_FILENAME = 'samples.h5'
FEATHER_EXT = '.feather'
ROWS_TO_READ = 1000000
MAX_ROWS_CHARTEVENTS = 330712483

skip_rows = 0

In [4]:
chartevents_columns = ["ROW_ID","SUBJECT_ID","HADM_ID","ICUSTAY_ID","ITEMID","CHARTTIME","STORETIME","CGID","VALUE","VALUENUM","VALUEUOM","WARNING","ERROR","RESULTSTATUS","STOPPED"]



# chartevents_dtype = {'ROW_ID':object,
# 'SUBJECT_ID':np.int64,
# 'HADM_ID':np.int64,
# 'ICUSTAY_ID':np.int64,
# 'ITEMID':np.int64,
# 'CHARTTIME':object,
# 'STORETIME':object,
# 'CGID':np.int64,
# 'VALUE':object,
# 'VALUENUM':np.int64,
# 'VALUEUOM':object,
# 'WARNING':np.int64,
# 'ERROR':np.int64,
# 'RESULTSTATUS':object,
# 'STOPPED':object}

chartevents_dtype = {'ROW_ID':str,
'SUBJECT_ID':np.float64,
'HADM_ID':np.float64,
'ICUSTAY_ID':np.float64,
'ITEMID':np.float64,
'CHARTTIME':str,
'STORETIME':str,
'CGID':str,
'VALUE':str,
'VALUENUM':str,
'VALUEUOM':str,
'WARNING':str,
'ERROR':str,
'RESULTSTATUS':str,
'STOPPED':str}

In [5]:
readmission = pd.read_csv(READMISSION_FILENAME).sort_values(['SUBJECT_ID','HADM_ID','ICUSTAY_ID']).reset_index(drop=True)

# d = readmission.dtypes.to_dict()
# d

In [6]:
icustay_list = readmission['ICUSTAY_ID'].unique().tolist()
icustay_list.sort()
# len(icustay_list)
division_index = np.array(range(icustay_list[0],icustay_list[-1],int(len(icustay_list)/300)))
division_index = np.append(division_index,icustay_list[-1]+1)
division_index = division_index.astype(str)
division_index = np.char.add(division_index,np.array(['_0']*len(division_index)).astype(str))
# division_index = np.apply_along_axis(lambda x: x+'_0',0,division_index)
len(division_index)

619

In [7]:
client = Client()
client

2023-05-02 12:05:02,883 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-8mvmdjo0', purging
2023-05-02 12:05:02,883 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-r3a2bvol', purging
2023-05-02 12:05:02,883 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-bk00mfsp', purging
2023-05-02 12:05:02,884 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-b7ooxnoj', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 62.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46597,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 62.57 GiB
Comm: tcp://127.0.0.1:42057,Total threads: 4
Dashboard: http://127.0.0.1:41667/status,Memory: 15.64 GiB
Nanny: tcp://127.0.0.1:39069,


In [8]:
df = dd.read_csv(CHARTEVENTS_FILENAME, dtype=chartevents_dtype ,assume_missing=True)

In [9]:
df = df.dropna(subset=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','CHARTTIME']).drop('ROW_ID',axis=1)

In [10]:
df['SUBJECT_ID'] = df.map_partitions(lambda x: x['SUBJECT_ID'].astype(np.int64))
df['HADM_ID'] = df.map_partitions(lambda x: x['HADM_ID'].astype(np.int64))
df['ICUSTAY_ID'] = df.map_partitions(lambda x: x['ICUSTAY_ID'].astype(np.int64))
df['CHARTTIME'] = df.map_partitions(lambda x: pd.to_datetime(x['CHARTTIME'], errors='coerce'))# , format='%Y-%m-%d %H:%M:%S')

/tmp/ipykernel_233207/3844895462.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CHARTTIME'] = df.map_partitions(lambda x: pd.to_datetime(x['CHARTTIME'], errors='coerce'))# , format='%Y-%m-%d %H:%M:%S')


In [11]:
# DO NOT set an index that is not unique. This will cause all sorts of issues that no one seems to have heard of having.
# df = df.set_index('ICUSTAY_ID',drop=True, sorted=False)

In [12]:
# len(df.divisions)
# (df.divisions)
df.info

<bound method DataFrame.info of Dask DataFrame Structure:
                SUBJECT_ID HADM_ID ICUSTAY_ID   ITEMID       CHARTTIME STORETIME    CGID   VALUE VALUENUM VALUEUOM WARNING   ERROR RESULTSTATUS STOPPED
npartitions=551                                                                                                                                        
                     int64   int64      int64  float64  datetime64[ns]    object  object  object   object   object  object  object       object  object
                       ...     ...        ...      ...             ...       ...     ...     ...      ...      ...     ...     ...          ...     ...
...                    ...     ...        ...      ...             ...       ...     ...     ...      ...      ...     ...     ...          ...     ...
                       ...     ...        ...      ...             ...       ...     ...     ...      ...      ...     ...     ...          ...     ...
                       ...    

In [13]:
valid_chartevents = df.map_partitions(lambda df: dd.multi.merge(readmission[['SUBJECT_ID','HADM_ID','ICUSTAY_ID']],df,'inner',['SUBJECT_ID','HADM_ID','ICUSTAY_ID']))
#valid_chartevents = df.map_partitions(lambda df: dd.multi.merge(readmission[['ICUSTAY_ID']],df,'inner',['ICUSTAY_ID']))

In [14]:
# This worked
# valid_chartevents.to_parquet(SAMPLES_DIR, engine="pyarrow", partition_on='ICUSTAY_ID', compute=True)

## Below fails to write to file. Shuffle fails again

In [15]:
valid_chartevents = valid_chartevents.sort_values(['ICUSTAY_ID', 'CHARTTIME'])

In [16]:
valid_chartevents['i'] = valid_chartevents.groupby('ICUSTAY_ID')['CHARTTIME'].cumcount().astype(int)
valid_chartevents['i'] = valid_chartevents['ICUSTAY_ID'].astype(str) + '_' + valid_chartevents['i'].astype(str)

In [17]:
indexed = valid_chartevents.set_index('i',drop=True, sorted=False, divisions=division_index.tolist())

In [18]:
indexed

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
npartitions=618,,,,,,,,,,,,,,
200001_0,int64,int64,int64,float64,datetime64[ns],object,object,object,object,object,object,object,object,object
200163_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299955_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300000_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [19]:
max_charttime = indexed.groupby('ICUSTAY_ID')['CHARTTIME'].max().rename('MAX_CHARTTIME')
max_charttime

Dask Series Structure:
npartitions=1
    datetime64[ns]
               ...
Name: MAX_CHARTTIME, dtype: datetime64[ns]
Dask Name: rename, 44 graph layers

In [20]:
indexed = dd.merge(indexed, max_charttime, 'left','ICUSTAY_ID')
indexed

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,MAX_CHARTTIME
npartitions=618,,,,,,,,,,,,,,,
200001_0,int64,int64,int64,float64,datetime64[ns],object,object,object,object,object,object,object,object,object,datetime64[ns]
200163_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299955_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300000_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Use indexed to find ICD9 Diagnosis?

## Keep only last 48 hours of ICU stay

## only keep items from chart events dims

In [24]:
# CE.repartition(division_index.tolist())

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,MAX_CHARTTIME
npartitions=618,,,,,,,,,,,,,,,
200001_0,int64,int64,int64,float64,datetime64[ns],object,object,object,object,object,object,object,object,object,datetime64[ns]
200163_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299955_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300000_0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [25]:
CE.to_parquet('data/last48hours/', engine="pyarrow", partition_on='ICUSTAY_ID', compute=True)

2023-05-02 11:56:10,186 - distributed.worker - WARNING - Compute Failed
Key:       ('shuffle-p2p-40d569075bc8827bb1ac7af8d07a225f', 547)
Function:  shuffle_unpack
args:      ('3a81921dff666befc00d856a3ca2d5a6', 547, 0)
kwargs:    {}
Exception: "RuntimeError('shuffle_unpack failed during shuffle 3a81921dff666befc00d856a3ca2d5a6')"

2023-05-02 11:56:10,188 - distributed.worker - WARNING - Compute Failed
Key:       ('shuffle-p2p-40d569075bc8827bb1ac7af8d07a225f', 548)
Function:  shuffle_unpack
args:      ('3a81921dff666befc00d856a3ca2d5a6', 548, 0)
kwargs:    {}
Exception: "RuntimeError('shuffle_unpack failed during shuffle 3a81921dff666befc00d856a3ca2d5a6')"

2023-05-02 11:56:10,220 - distributed.worker - WARNING - Compute Failed
Key:       ('shuffle-p2p-40d569075bc8827bb1ac7af8d07a225f', 550)
Function:  shuffle_unpack
args:      ('3a81921dff666befc00d856a3ca2d5a6', 550, 0)
kwargs:    {}
Exception: "RuntimeError('shuffle_unpack failed during shuffle 3a81921dff666befc00d856a3ca2d5a6')"

2

RuntimeError: shuffle_unpack failed during shuffle 3a81921dff666befc00d856a3ca2d5a6

In [ ]:
# pdf[100:120]